# load, predict

In [1]:
import cv2
import os
import time
import numpy as np
import tensorflow as tf
import time
import collections
import PIL
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model
from tqdm import tqdm_notebook as tqdm
import tensorflow.contrib.tensorrt as trt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
output_names = ['predictions/Softmax']
input_names = ['input_1']

import tensorflow as tf


def get_frozen_graph(graph_file):
    """Read Frozen Graph file from disk."""
    with tf.gfile.FastGFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def


trt_graph = get_frozen_graph('./model/trt_graph.pb')

# Create session and load graph
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_sess = tf.Session(config=tf_config)

Instructions for updating:
Use tf.gfile.GFile.


In [3]:
tf.import_graph_def(trt_graph, name='')

In [4]:
# Get graph input size
for node in trt_graph.node:
    if 'input_' in node.name:
        size = node.attr['shape'].shape
        image_size = [size.dim[i].size for i in range(1, 4)]
        break
print("image_size: {}".format(image_size))


# input and output tensor names.
input_tensor_name = input_names[0] + ":0"
output_tensor_name = output_names[0] + ":0"

print("input_tensor_name: {}\noutput_tensor_name: {}".format(
    input_tensor_name, output_tensor_name))



image_size: [71, 71, 3]
input_tensor_name: input_1:0
output_tensor_name: predictions/Softmax:0


In [5]:
output_tensor = tf_sess.graph.get_tensor_by_name(output_tensor_name)

In [6]:
output_tensor

<tf.Tensor 'predictions/Softmax:0' shape=<unknown> dtype=float32>

In [27]:
folder = "/home/jetson/datasets/colours/1"
total = np.empty([0, 71, 71, 3])
dim = (71, 71)
for i, file in enumerate(tqdm(os.listdir(folder))):
    if i > 200000:
        break
    x = image.load_img(folder + "/" + file, target_size=dim)
    x = np.array(x)
    x = x/255.0
    x = np.expand_dims(x, axis=0)
    total = np.concatenate((total, x))

In [28]:
total.shape

(1579, 71, 71, 3)

In [29]:
total[0:1][0].shape

(71, 71, 3)

In [30]:
img = total[0:1][0]
img.shape

(71, 71, 3)

In [31]:
'''
for i in range(10):
    img = total[i:i+1][0]
    cv2.imshow('image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
'''

In [32]:
start = time.time()
predictions = np.empty((1, 6))
for i in tqdm(range(total.shape[0])):
    #t = np.transpose(total[i:i+1], axes=(0, 2, 1, 3))
    feed_dict = {
    input_tensor_name: total[i:i+1]
    }
    preds = tf_sess.run(output_tensor, feed_dict)
    predictions = np.concatenate((predictions, preds))
end = time.time()
print(end - start)


7.641301870346069


In [33]:
predictions = predictions[1:]

In [34]:
predictions.shape

(1579, 6)

In [35]:
c = collections.Counter(np.argmax(predictions[:, :], axis=1))
c

Counter({1: 1455, 2: 39, 3: 8, 0: 57, 4: 8, 5: 12})